<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 7.7 MB/s 
     |████████████████████████████████| 166 kB 64.5 MB/s 
     |████████████████████████████████| 182 kB 48.8 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 50.0 MB/s 
     |████████████████████████████████| 162 kB 66.6 MB/s 
     |████████████████████████████████| 162 kB 70.4 MB/s 
     |████████████████████████████████| 158 kB 72.3 MB/s 
     |████████████████████████████████| 157 kB 64.1 MB/s 
     |████████████████████████████████| 157 kB 72.8 MB/s 
     |████████████████████████████████| 157 kB 76.9 MB/s 
     |████████████████████████████████| 157 kB 70.6 MB/s 
     |████████████████████████████████| 157 kB 74.4 MB/s 
     |████████████████████████████████| 157 kB 77.1 MB/s 
     |████████████████████████████████| 157 kB 68.6 MB/s 
     |████████████████████████████████| 156 kB 72.3 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: ht

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.


In [11]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
class_names == test_class_names
print(class_names)
num_classes = len(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [12]:
xception = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_xception=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  xception,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
xception.trainable=False
model_xception.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 xception (Functional)       (None, None, None, 2048)  20861480  
                                                                 
 flatten_3 (Flatten)         (None, 100352)            0         
                                                                 
 dense_3 (Dense)             (None, 32)                3211296   
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 5)                 165       
                                                                 
Total params: 24,072,941
Trainable params: 3,211,461
N

In [13]:
model_xception.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [14]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "Xception",
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [15]:
%%time
epochs=40
history = model_xception.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 0.8185 - accuracy: 0.7614

wandb: Adding directory to artifact (/content/wandb/run-20221104_114436-3kwa6ehy/files/model-best)... Done. 1.1s


22/22 [==============================] - 166s 5s/step - loss: 0.8185 - accuracy: 0.7614 - val_loss: 0.2851 - val_accuracy: 0.8971
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.2380 - accuracy: 0.9375

wandb: Adding directory to artifact (/content/wandb/run-20221104_114436-3kwa6ehy/files/model-best)... Done. 1.0s


...

22/22 [==============================] - 22s 1s/step - loss: 0.0795 - accuracy: 0.9759 - val_loss: 0.0821 - val_accuracy: 0.9600
Epoch 8/40
22/22 [==============================] - 5s 189ms/step - loss: 0.0504 - accuracy: 0.9815 - val_loss: 0.1132 - val_accuracy: 0.9657
Epoch 9/40
22/22 [==============================] - ETA: 0s - loss: 0.0479 - accuracy: 0.9773

wandb: Adding directory to artifact (/content/wandb/run-20221104_114436-3kwa6ehy/files/model-best)... Done. 1.0s


22/22 [==============================] - 22s 1s/step - loss: 0.0479 - accuracy: 0.9773 - val_loss: 0.0632 - val_accuracy: 0.9771
Epoch 10/40
22/22 [==============================] - 5s 189ms/step - loss: 0.0358 - accuracy: 0.9858 - val_loss: 0.0699 - val_accuracy: 0.9771
Epoch 11/40
22/22 [==============================] - 5s 189ms/step - loss: 0.0405 - accuracy: 0.9858 - val_loss: 0.1285 - val_accuracy: 0.9543
Epoch 12/40
22/22 [==============================] - 5s 192ms/step - loss: 0.0560 - accuracy: 0.9787 - val_loss: 0.0959 - val_accuracy: 0.9714
Epoch 13/40
22/22 [==============================] - 5s 191ms/step - loss: 0.0341 - accuracy: 0.9886 - val_loss: 0.0705 - val_accuracy: 0.9714
Epoch 14/40
22/22 [==============================] - 5s 192ms/step - loss: 0.0466 - accuracy: 0.9830 - val_loss: 0.0782 - val_accuracy: 0.9829
Epoch 15/40
22/22 [==============================] - 5s 193ms/step - loss: 0.0292 - accuracy: 0.9872 - val_loss: 0.1433 - val_accuracy: 0.9714
Epoch 16/40
2

wandb: Adding directory to artifact (/content/wandb/run-20221104_114436-3kwa6ehy/files/model-best)... Done. 1.0s


22/22 [==============================] - 22s 1s/step - loss: 0.0575 - accuracy: 0.9915 - val_loss: 0.0336 - val_accuracy: 0.9829
Epoch 19/40
22/22 [==============================] - 5s 192ms/step - loss: 0.0324 - accuracy: 0.9872 - val_loss: 0.1147 - val_accuracy: 0.9714
Epoch 20/40
22/22 [==============================] - 5s 219ms/step - loss: 0.0165 - accuracy: 0.9972 - val_loss: 0.0788 - val_accuracy: 0.9829
Epoch 21/40
22/22 [==============================] - 5s 198ms/step - loss: 0.0170 - accuracy: 0.9901 - val_loss: 0.0717 - val_accuracy: 0.9829
Epoch 22/40
22/22 [==============================] - 5s 190ms/step - loss: 0.0187 - accuracy: 0.9901 - val_loss: 0.1026 - val_accuracy: 0.9829
Epoch 23/40
22/22 [==============================] - 5s 189ms/step - loss: 0.0680 - accuracy: 0.9815 - val_loss: 0.2423 - val_accuracy: 0.9486
Epoch 24/40
22/22 [==============================] - 5s 191ms/step - loss: 0.0749 - accuracy: 0.9815 - val_loss: 0.2159 - val_accuracy: 0.9657
Epoch 25/40
2

In [16]:
wandb.finish()

accuracy,▁▆▇▇▇▇▇█▇██▇████████████▇▇▇█████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▅▃▅▅▆▇██▆▇▇█▇█▆█▇███▅▇▆▆▇██▇▇▆▇▆▆▄▆▆▆▆
val_loss,▅▃▃▅▄▂▂▂▂▂▃▂▂▂▃▂▄▁▂▂▂▂▅▄▃▄▂▂▄▄▄▃▂▃▃█▃▄▄▅
accuracy,0.99006
best_epoch,17
best_val_loss,0.03355
epoch,39
loss,0.03017
val_accuracy,0.96
